# TV Script Generation

In this project, an own version of [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs will be created. Parts of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons is used.  The Neural Network built will generate a new, "fake" TV script, based on patterns it recognises in this training data.


In [1]:
# Load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

In [2]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
from torch.nn.utils import clip_grad_norm_
import numpy as np
import helper
import problem_unittests as tests
import re
from collections import Counter

## Explore the Data
Looking around `view_line_range` to view different parts of the data. This gives a sense of the data that will be worked with. It can be seen, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [3]:
view_line_range = (0, 10)

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, first need to transform the words to ids.  In this function, two dictionaries are created:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

These dictionaries are returned in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [4]:
def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    vocab = tuple(set(text))
    int_to_vocab = dict(enumerate(vocab))
    vocab_to_int = {ch: ii for ii, ch in int_to_vocab.items()}
    return (vocab_to_int, int_to_vocab)

tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
The script will be splitting into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Therefore, the function `token_lookup` is implemented to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  A dictionary is created for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Confusion to prevent value as a word needs to be accounted for; for example, instead of using the value "dash", something like "||dash||" can be used.

In [5]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """       
    return {
        '.': '||Period||',
        ',': '||Comma||',
        '"': '||Quotation_Mark||',
        ';': '||Semicolon||',
        '!': '||Exclamation_Mark||',
        '?': '||Question_Mark||',
        '(': '||Left_Parentheses||',
        ')': '||Right_Parentheses||',
        '-': '||Dash||',
        '\n': '||Return||',
    }

tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. The code for `preprocess_and_save_data` in the `helpers.py` file is used. 

In [6]:
# Pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is the first checkpoint. If ever decided to come back to this notebook or have to restart the notebook, it can be started from here. The preprocessed data has been saved to disk.

In [7]:
int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions will be built.

### Check Access to GPU

In [8]:
# Check for GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train the neural network.')

## Input
Let's start with the preprocessed input data. [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) is used to provide a known format to the dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

Data can be created with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
The `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes is implemented.

>DataLoader can be used to batch words. `feature_tensors` and `target_tensors` of the correct size are created and content for a given `sequence_length`.

For example, say this is the input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

The first `feature_tensor` could contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` would just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [9]:
def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    feature_tensors = []
    target_tensors = []
    
    for i in range(len(words) - sequence_length - 1):
        feature_tensors.append(words[i:i+sequence_length])
        target_tensors.append(words[i+sequence_length])

    feature_tensors = torch.Tensor(feature_tensors)
    target_tensors = torch.Tensor(target_tensors)

    data = TensorDataset(feature_tensors, target_tensors)
    data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)
    return data_loader

### Test the dataloader 

A batching function will be tested here.

Below, some test is generating text data and defining a dataloader using the function defined above. Then, some sample batch of inputs `sample_x` and targets `sample_y` are obtained from the dataloader.

The code would return something like the following (likely in a different order, if data is shuffled):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
sample_x will be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y has one dimension: batch_size (10). 

### Values

Notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [10]:
# Test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[  5.,   6.,   7.,   8.,   9.],
        [ 13.,  14.,  15.,  16.,  17.],
        [ 15.,  16.,  17.,  18.,  19.],
        [ 24.,  25.,  26.,  27.,  28.],
        [ 28.,  29.,  30.,  31.,  32.],
        [  3.,   4.,   5.,   6.,   7.],
        [ 37.,  38.,  39.,  40.,  41.],
        [  8.,   9.,  10.,  11.,  12.],
        [ 23.,  24.,  25.,  26.,  27.],
        [ 40.,  41.,  42.,  43.,  44.]])

torch.Size([10])
tensor([ 10.,  18.,  20.,  29.,  33.,   8.,  42.,  13.,  28.,  45.])


---
## Build the Neural Network
An RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module) will be implemented. A GRU or an LSTM is possible chosen options. To complete the RNN, the following functions are implemented for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, only want to output the word scores for a single, most likely, next word.

### Notes

1. Stack the outputs of the lstm to pass to the fully-connected layer. This can be done with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. Obtain the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [11]:
class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        
        # Set class variables
        self.vocab_size = vocab_size
        self.output_size = output_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.dropout = dropout
        
        # Define model layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        self.linear = nn.Linear(hidden_dim, output_size)
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """        
        nn_input = nn_input.long()
        batch_size = nn_input.size(0)
        lstm_output, hidden = self.lstm(self.embedding(nn_input), hidden)
        lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)
        output = self.linear(lstm_output)
        output = output.view(batch_size, -1, self.output_size)
        out = output[:, -1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_dim)).to(device),
               Variable(torch.zeros(self.n_layers, batch_size, self.hidden_dim)).to(device))


tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

The RNN class implemented will be used to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

It returns the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. By computing it, the loss is gained, as usual, and calling `loss.item()`.

**If a GPU is available, data should moved to to that GPU device, here.**

In [12]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    # Move data to GPU, if available
    inp = inp.to(device)
    target = target.to(device)
    hidden = tuple([layer.data.to(device) for layer in hidden])

    optimizer.zero_grad()
    
    output, hidden = rnn(inp, hidden)
    
    # Perform backpropagation and optimization
    target = target.long()
    loss = criterion(output, target)
    loss.backward()
    
    clip_factor = 10
    clip_grad_norm_(rnn.parameters(), clip_factor)
    optimizer.step()
    
    # Return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. This parameter is set along with other parameters in the next section.

In [13]:
def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # Initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # Make sure to iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # Forward, Back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # Record loss
            batch_losses.append(loss)

            # Printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # Returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [14]:
# Data params
# Sequence Length
sequence_length = 15  # Number of words in a sequence
# Batch Size
batch_size = 256

# Data loader
train_loader = batch_data(int_text, sequence_length, batch_size)

In [15]:
# Training parameters
# Number of Epochs
num_epochs = 10
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = len(vocab_to_int)
# Embedding Dimension
embedding_dim = 256
# Hidden Dimension
hidden_dim = 256
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 500

### Train
In the next cell, the neural network is trained on the pre-processed data.  If a good loss isn't obtained easily, the hyperparameters may need to be changed. In general, better results are obtained with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **Aim should be for a loss less than 3.5.** 

Different sequence lengths should be experimented, which determine the size of the long range dependencies that a model can learn.

In [16]:
# Create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# Defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# Saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 10 epoch(s)...
Epoch:    1/10    Loss: 5.487971757888794

Epoch:    1/10    Loss: 4.741044304847717

Epoch:    1/10    Loss: 4.528067712783813

Epoch:    1/10    Loss: 4.388593851089477

Epoch:    1/10    Loss: 4.2928066825866695

Epoch:    1/10    Loss: 4.234217581748962

Epoch:    2/10    Loss: 4.1187964103570796

Epoch:    2/10    Loss: 4.030122438907624

Epoch:    2/10    Loss: 4.0007367186546325

Epoch:    2/10    Loss: 3.9840150423049927

Epoch:    2/10    Loss: 3.9588888349533082

Epoch:    2/10    Loss: 3.9276961307525635

Epoch:    3/10    Loss: 3.8665127131512493

Epoch:    3/10    Loss: 3.7920917444229127

Epoch:    3/10    Loss: 3.7802991976737976

Epoch:    3/10    Loss: 3.7877189717292787

Epoch:    3/10    Loss: 3.7715495085716246

Epoch:    3/10    Loss: 3.779752869129181

Epoch:    4/10    Loss: 3.7091773116007083

Epoch:    4/10    Loss: 3.6477814621925355

Epoch:    4/10    Loss: 3.637330244064331

Epoch:    4/10    Loss: 3.657658308506012

Epoch:    4/1

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved


### Question: How was the model hyperparameters decided? 
For example, was a different sequence_lengths tried and find that one size made the model converge faster? What about  hidden_dim and n_layers; how were those decided?

**Answer:** 
* `vocab_size` and `output_size` are number of unique words in the vocabulary.
* `embedding_dim` and `hidden_dim` are 256, value which are of the power of 2. The recommendation is in the range 200 and 500. 
* `learning_rate` - 0.001, 0.005, 0.01, 0.05, were tried and 0.001 provided the best result. 
* `num_epochs` - 10 was chosen but between 6-7 would have been sufficient too. The extra epoch would just increase the accuracy, but it had to be ensured the model is not being overtrained. 10 was satisfactory for this reason.
* `batch_size` - The value is high enough to train efficiently while also avoiding GPU memory overload.
* `sequence_length` - At 20, the model was taking longer to converge, therefore 15 works well.

---
# Checkpoint

After running the above training cell, the model will be saved by name, `trained_rnn`, and if saved notebook progress, **the process can be paused here and come back to this code at another time**. Resume the progress by running the next cell, which will load in the word:id dictionaries _and_ load in the saved model by name!

In [17]:
_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, it is used to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. The `generate` function is used to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [18]:
def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # Create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # Initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # Get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # Get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # Use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # Select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # Retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # The generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # Return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script needed to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

The prime word is set to _any word_ in the dictionary, but it's best to start with a name for generating a TV script. (It can also start with any other names found in the original text file!)

In [38]:
# Run the cell multiple times to get different results!
gen_length = 400 # Modify the length to preference
prime_word = 'jerry' # Name for starting the script

pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:57: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


jerry: he addresses her glistening spell the bullseye.

jerry: i can't believe that i said.

kramer:(to jerry) yeah!

kramer: oh, you know i was just wondering if i can have a mantel, and you could have gone back in the building.

jerry: no, i don't think we should get the money.

elaine: well, i didn't want to hear that.

elaine:(to jerry) you know, it's a lot better than me, but i'm going out to get a new bedspread.

jerry:(to elaine) yeah, well, i think i can do that.

elaine: well, what do you want?

george: i know...

elaine: what?!

kramer:(smiling) well, i don't think so.

george: you didn't know that?!?

kramer: yeah.

elaine: yeah, yeah.

george: i know... i don't know what it costs.

george: i don't know, but i think you can see me.

george: well, i was thinking about that.

george: you don't think i should go.

jerry: i know, i can't believe it! i can't do it!!

jerry: what?

george: i can't believe that you can get me in the car and then i don't know what i am.

jerry: you 

#### Save the favorite scripts

Once a script that is liked (or found interesting) is obtained, save it to a text file!

In [39]:
# Save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
The TV script may make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

It can be seen that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, a smaller vocabulary has to be used (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for the purposes; for script generation more than 1 MB of text is needed, generally. 